# Differential private analysis

In [38]:
import opendp.prelude as dp
dp.enable_features("contrib")

In [39]:
#import data from csv file
with open("C:\\Users\kshub\\OneDrive\\Documents\\PET_phase_2\\Technical_Phase_Data\\technical_phase_data.csv") as f:
    data=f.read()
print(data[0:1000])

<>:2: SyntaxWarning: invalid escape sequence '\k'
<>:2: SyntaxWarning: invalid escape sequence '\k'
C:\Users\kshub\AppData\Local\Temp\ipykernel_14328\752028492.py:2: SyntaxWarning: invalid escape sequence '\k'
  with open("C:\\Users\kshub\\OneDrive\\Documents\\PET_phase_2\\Technical_Phase_Data\\technical_phase_data.csv") as f:


ID,date,merch_category,merch_postal_code,transaction_type,spendamt,nb_transactions
1,2019-01-01,Grocery Stores/Supermarkets,8700000,ONLINE,11238.128449812339,160
1,2019-01-01,Grocery Stores/Supermarkets,0500034,ONLINE,12848.165220827457,183
2,2019-01-01,Grocery Stores/Supermarkets,110621,ONLINE,12116.165569209737,173
3,2019-01-01,Hotels/Motels,8900000,OFFLINE,7745.9988789353365,38
4,2019-01-01,Restaurants,111941,OFFLINE,6927.424754451023,173
4,2019-01-01,Restaurants,111111,OFFLINE,7513.0969230991095,187
4,2019-01-01,Restaurants,111061,OFFLINE,6337.253136719919,158
5,2019-01-01,Restaurants,8780000,ONLINE,6596.213194976057,164
5,2019-01-01,Restaurants,111321,ONLINE,7109.564110740599,177
6,2019-01-01,General Retail Stores,70675,ONLINE,8150.981488290598,81
7,2019-01-01,Drug Stores/Pharmacies,9460000,ONLINE,9029.354227509248,300
8,2019-01-01,Drug Stores/Pharmacies,70374,OFFLINE,9239.46832964432,307
9,2019-01-01,"Utilities: Electric, Gas, Water",70253,ONLINE,7241.559244978171,48
10,2019-01-0

In [40]:
privacy_unit=dp.unit_of(contributions=1)
privacy_loss=dp.loss_of(epsilon=1.0)

In [41]:
context=dp.Context.compositor(
    data=data,
    privacy_unit=privacy_unit,
    privacy_loss=privacy_loss,
    split_evenly_over=10
)

In [42]:
col_names=['ID', 'date', 'merch_category', 'merch_postal_code', 'transaction_type',
       'spendamt', 'nb_transactions']
merch_types=['Grocery Stores/Supermarkets', 'Hotels/Motels',
       'Restaurants', 'General Retail Stores', 'Drug Stores/Pharmacies',
       'Hospitals','Utilities: Electric, Gas, Water',
       'Bars/Discotheques', 'Computer Network/Information Services',
       'Airlines']

In [55]:
count_query=(
    context.query().split_dataframe(',', col_names=col_names).select_column("merch_category",str).count_by_categories(merch_types,null_category=False).laplace()
)
count_query.release()

[626775, 284458, 942172, 690929, 436603, 347781, 19, 386652, 177432, 93010]

In [61]:
Bridge_count_query=(
    context.query().split_dataframe(",",col_names=col_names).df_is_equal("merch_category", "Grocery Stores/Supermarkets").subset_by("merch_category", keep_columns=["ID"]).select_column("ID",str).count().laplace()
)
Bridge_count_query.release()

626808

# pandas analysis

In [44]:
import pandas as pd
import numpy as np
df=pd.read_csv("C:\\Users\kshub\\OneDrive\\Documents\\PET_phase_2\\Technical_Phase_Data\\technical_phase_data.csv",names=col_names)

<>:3: SyntaxWarning: invalid escape sequence '\k'
<>:3: SyntaxWarning: invalid escape sequence '\k'
C:\Users\kshub\AppData\Local\Temp\ipykernel_14328\1925529077.py:3: SyntaxWarning: invalid escape sequence '\k'
  df=pd.read_csv("C:\\Users\kshub\\OneDrive\\Documents\\PET_phase_2\\Technical_Phase_Data\\technical_phase_data.csv",names=col_names)
C:\Users\kshub\AppData\Local\Temp\ipykernel_14328\1925529077.py:3: DtypeWarning: Columns (0,3,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("C:\\Users\kshub\\OneDrive\\Documents\\PET_phase_2\\Technical_Phase_Data\\technical_phase_data.csv",names=col_names)


In [45]:
df.head()

,ID,date,merch_category,merch_postal_code,transaction_type,spendamt,nb_transactions
0,ID,date,merch_category,merch_postal_code,transaction_type,spendamt,nb_transactions
1,1,2019-01-01,Grocery Stores/Supermarkets,8700000,ONLINE,11238.128449812339,160
2,1,2019-01-01,Grocery Stores/Supermarkets,0500034,ONLINE,12848.165220827457,183
3,2,2019-01-01,Grocery Stores/Supermarkets,110621,ONLINE,12116.165569209737,173
4,3,2019-01-01,Hotels/Motels,8900000,OFFLINE,7745.9988789353365,38


In [46]:
df['merch_category'].value_counts()

merch_category
Restaurants                              942172
General Retail Stores                    690954
Grocery Stores/Supermarkets              626791
Drug Stores/Pharmacies                   436601
Bars/Discotheques                        386650
Hospitals                                347776
Hotels/Motels                            284449
Utilities: Electric, Gas, Water          194370
Computer Network/Information Services    177441
Airlines                                  93005
merch_category                                1
Name: count, dtype: int64

In [47]:
df["merch_category"].unique()

array(['merch_category', 'Grocery Stores/Supermarkets', 'Hotels/Motels',
       'Restaurants', 'General Retail Stores', 'Drug Stores/Pharmacies',
       'Utilities: Electric, Gas, Water', 'Hospitals',
       'Bars/Discotheques', 'Computer Network/Information Services',
       'Airlines'], dtype=object)

In [48]:
df.columns

Index(['ID', 'date', 'merch_category', 'merch_postal_code', 'transaction_type',
       'spendamt', 'nb_transactions'],
      dtype='object')